# EDA


- The os module has a perfect method to list files in a directory.
- Pandas json normalize could work here but is not necessary to convert the JSON data to a dataframe.
- You may need a nested for-loop to access each sale!
- We've put a lot of time into creating the structure of this repository, and it's a good example for future projects.  In the file functions_variables.py, there is an example function that you can import and use.  If you have any variables, functions or classes that you want to make, they can be put in the functions_variables.py file and imported into a notebook.  Note that only .py files can be imported into a notebook. If you want to import everything from a .py file, you can use the following:
```python
from functions_variables import *
```
If you just import functions_variables, then each object from the file will need to be prepended with "functions_variables"\
Using this .py file will keep your notebooks very organized and make it easier to reuse code between notebooks.

In [21]:
# (this is not an exhaustive list of libraries)
import pandas as pd
import numpy as np
import os
import json
from pprint import pprint
from functions_variables import encode_tags
from os import listdir
from os.path import join, isfile


## Data Importing

In [29]:
# load one file first to see what type of data you're dealing with and what attributes it has

In [30]:
# loop over all files and put them into a dataframe
import os
import pandas as pd

# Directory containing your JSON files
json_directory = 'C:/Users/spencer.fargey/LighthouseLabs/tdsf-midterm/data'  # Replace with the actual path to your directory

# Initialize an empty list to store DataFrames from each JSON file
dataframes = []

# Function to load and process each JSON file
def load_json_file(fname):
    """
    This function loads a JSON file (assumed to be ASCII-readable)
    and extracts the 'results' from the 'data' field.
    """
    with open(fname, 'r') as f:
        try:
            data_json = json.load(f)
            # Check if 'data' and 'results' exist in the loaded JSON
            if 'data' in data_json and 'results' in data_json['data']:
                df = pd.json_normalize(data_json['data']['results'])
            else:
                raise KeyError("'data' or 'results' field not found in the JSON file")
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON in file {fname}: {e}")
            return None
        except KeyError as e:
            print(f"Key error in file {fname}: {e}")
            return None
    
    return df

# Loop through all files in the directory
for filename in os.listdir(json_directory):
    if filename.endswith(".json"):  # Check if the file is a JSON file
        file_path = os.path.join(json_directory, filename)  # Full file path
        df = load_json_file(file_path)  # Call load_json_file with fname
        if df is not None:  # If the DataFrame is valid, append to list
            dataframes.append(df)

# Combine all DataFrames into one
if dataframes:
    final_df = pd.concat(dataframes, ignore_index=True)
    print(final_df)
else:
    print("No valid JSON files were processed.")


FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/spencer.fargey/LighthouseLabs/tdsf-midterm/data'

In [27]:
# Save the dataframe to a CSV file
output_csv_path = 'C:/Users/spencer.fargey/LighthouseLabs/tdsf-midterm/Housing_Data.csv'
final_df.to_csv(output_csv_path, index=False)

print(f"CSV saved to: {output_csv_path}")


NameError: name 'final_df' is not defined

## Data Cleaning and Wrangling

At this point, ensure that you have all sales in a dataframe.
- Take a quick look at your data (i.e. `.info()`, `.describe()`) - what do you see?
- Is each cell one value, or do some cells have lists?
- What are the data types of each column?
- Some sales may not actually include the sale price (target).  These rows should be dropped.
- There are a lot of NA/None values.  Should these be dropped or replaced with something?
    - You can drop rows or use various methods to fills NA's - use your best judgement for each column 
    - i.e. for some columns (like Garage), NA probably just means no Garage, so 0
- Drop columns that aren't needed
    - Don't keep the list price because it will be too close to the sale price. Assume we want to predict the price of houses not yet listed

In [236]:
# load data here, data already concatenated into the csv in previous step
# re import libraries so you can run code from here, and not re-create the original csv file
import pandas as pd
import numpy as np
import os
import json
from pprint import pprint
from functions_variables import encode_tags
from os import listdir
from os.path import join, isfile

df = pd.read_csv("/Users/thomasdoherty/Desktop/tdsf-midterm/Housing_Data.csv") # loading the data from the csv file compiled for Thomas' computer

#df = pd.read_csv("/Users/spencer.fargey/Lighthouselabs/tdsf-midterm/Housing_Data.csv") # loading the data from the csv file compiled for Spencers Computer


df.head(5) # make sure it's loaded correctly

,last_update_date,tags,permalink,status,list_date,open_houses,branding,list_price,property_id,photos,community,virtual_tours,listing_id,price_reduced_amount,matterport,primary_photo.href,source.plan_id,source.agents,source.spec_id,source.type,description.year_built,description.baths_3qtr,description.sold_date,description.sold_price,description.baths_full,description.name,description.baths_half,description.lot_sqft,description.sqft,description.baths,description.sub_type,description.baths_1qtr,description.garage,description.stories,description.beds,description.type,lead_attributes.show_contact_an_agent,flags.is_new_construction,flags.is_for_rent,flags.is_subdivision,flags.is_contingent,flags.is_price_reduced,flags.is_pending,flags.is_foreclosure,flags.is_plan,flags.is_coming_soon,flags.is_new_listing,products.brand_name,other_listings.rdc,location.address.postal_code,location.address.state,location.address.coordinate.lon,location.address.coordinate.lat,location.address.city,location.address.state_code,location.address.line,location.street_view_url,location.county.fips_code,location.county.name,primary_photo,source,products,location.address.coordinate,other_listings,community.advertisers,community.description.name,location.county
0,2023-09-19T20:52:50Z,"['carport', 'community_outdoor_space', 'cul_de...",9453-Herbert-Pl_Juneau_AK_99801_M90744-30767,sold,2023-06-29T21:16:25.000000Z,NaN,"[{'name': 'EXP Realty LLC - Southeast Alaska',...",554950.0,9.074431e+09,"[{'tags': [{'label': 'house_view', 'probabilit...",NaN,NaN,2.957242e+09,45000.0,False,https://ap.rdcpix.com/07097d34c98a59ebb7996889...,NaN,[{'office_name': 'EXP Realty LLC - Southeast A...,NaN,mls,1963.0,NaN,2023-09-18,NaN,2.0,NaN,NaN,10454.0,1821.0,2.0,NaN,NaN,1.0,NaN,3.0,single_family,True,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,False,basic_opt_in,"[{'listing_id': '2957241843', 'listing_key': N...",99801.0,Alaska,-134.59372,58.36395,Juneau,AK,9453 Herbert Pl,https://maps.googleapis.com/maps/api/streetvie...,NaN,Juneau,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,8477-Thunder-Mountain-Rd_Juneau_AK_99801_M9424...,sold,NaN,NaN,"[{'name': None, 'photo': None, 'type': 'Office'}]",NaN,9.424984e+09,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-08-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'listing_id': '2958935271', 'listing_key': N...",99801.0,Alaska,NaN,NaN,Juneau,AK,8477 Thunder Mountain Rd,https://maps.googleapis.com/maps/api/streetvie...,NaN,Juneau,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,4515-Glacier-Hwy_Juneau_AK_99801_M94790-68516,sold,NaN,NaN,"[{'name': None, 'photo': None, 'type': 'Office'}]",NaN,9.479069e+09,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-08-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'listing_id': '2958935192', 'listing_key': N...",99801.0,Alaska,NaN,NaN,Juneau,AK,4515 Glacier Hwy,https://maps.googleapis.com/maps/api/streetvie...,NaN,Juneau,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,17850-Point-Stephens-Rd_Juneau_AK_99801_M98793...,sold,NaN,NaN,"[{'name': None, 'photo': None, 'type': 'Office'}]",NaN,9.879332e+09,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-08-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'listing_id': '2958925235', 'listing_key': N...",99801.0,Alaska,NaN,NaN,Juneau,AK,17850 Point Stephens Rd,https://maps.googleapis.com/maps/api/streetvie...,NaN,Juneau,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,9951-Stephen-Richards-Memorial-Dr_Juneau_AK_99...,sold,NaN,NaN,"[{'name': None, 'photo': None, 'type': 'Office'}]",NaN,9.521640e+09,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-08-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'listing_id': '2958924367', 'listing_key': N...",99801.0,Alaska,NaN,NaN,Juneau,AK,9951 Stephen Richards Memorial Dr,https://maps.goog

In [237]:
# basic exploration of the df
df.shape # 67 possible variables and 8.2k records

(8191, 67)

In [238]:
df.info() # lots of missing values, some completely full columns, some completely empty columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8191 entries, 0 to 8190
Data columns (total 67 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   last_update_date                       8125 non-null   object 
 1   tags                                   7638 non-null   object 
 2   permalink                              8159 non-null   object 
 3   status                                 8159 non-null   object 
 4   list_date                              7752 non-null   object 
 5   open_houses                            0 non-null      float64
 6   branding                               8159 non-null   object 
 7   list_price                             7721 non-null   float64
 8   property_id                            8159 non-null   float64
 9   photos                                 7403 non-null   object 
 10  community                              0 non-null      float64
 11  virt

From just looking at the info above there are **lots of null values**. There doesn't seem any point in getting descriptive statistics until we remove all the completely empty numerical columns. 

Check all columns for completely null

In [239]:
total = df.shape[0] # total number of rows - 8191

# how many missing values are there in each column?
null_counts = df.isnull().sum() # there are many columns with 8191 nulls - the entire column is null

# get a list of completely null columns to drop
completely_null_cols = null_counts[null_counts == total].index.tolist()

df[completely_null_cols].head(3) # these columns are completely null

,open_houses,community,description.name,description.baths_1qtr,flags.is_new_construction,flags.is_for_rent,flags.is_subdivision,flags.is_contingent,flags.is_pending,flags.is_plan,flags.is_coming_soon,primary_photo,source,products,location.address.coordinate,other_listings,location.county
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [240]:
df[completely_null_cols].shape # there are 17 columns that are completely null

(8191, 17)

In [241]:
# drop the completely null columns
df = df.drop(columns=completely_null_cols)

We have removed the 17 completely null columns (such as flag.is_subdivision, flag.is_for_rent, location.county...) so now there are 50 remaining columns.

I will look at partially null numerical columns from here and then the categorical columns.

In [242]:
# how many features are numerical and how many are categorical?
numerical = df.select_dtypes(include=[np.number]).columns
categorical = df.select_dtypes(include=['object']).columns

df[numerical].head(5) # there are 37 numerical features

,list_price,property_id,listing_id,price_reduced_amount,source.plan_id,description.year_built,description.baths_3qtr,description.sold_price,description.baths_full,description.baths_half,description.lot_sqft,description.sqft,description.baths,description.garage,description.stories,description.beds,location.address.postal_code,location.address.coordinate.lon,location.address.coordinate.lat,location.county.fips_code
0,554950.0,9.074431e+09,2.957242e+09,45000.0,NaN,1963.0,NaN,NaN,2.0,NaN,10454.0,1821.0,2.0,1.0,NaN,3.0,99801.0,-134.59372,58.36395,NaN
1,NaN,9.424984e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,99801.0,NaN,NaN,NaN
2,NaN,9.479069e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,99801.0,NaN,NaN,NaN
3,NaN,9.879332e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,99801.0,NaN,NaN,NaN
4,NaN,9.521640e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,99801.0,NaN,NaN,NaN


In [243]:
df[numerical].shape # there are 20 numerical features

(8191, 20)

In [244]:
# calculate the initial proportion of null entries in each column
null_proportion = df.isnull().sum() / total * 100

round(null_proportion.sort_values(ascending=False), 2)

community.description.name               99.94
source.plan_id                           99.94
source.spec_id                           99.94
community.advertisers                    99.94
flags.is_foreclosure                     99.49
description.baths_3qtr                   93.09
virtual_tours                            83.51
description.sub_type                     82.58
description.baths_half                   72.15
flags.is_price_reduced                   69.67
price_reduced_amount                     69.67
description.garage                       45.70
description.stories                      23.57
description.sold_price                   18.01
description.lot_sqft                     14.65
description.baths_full                   10.74
description.year_built                   10.68
description.sqft                         10.60
primary_photo.href                        9.62
photos                                    9.62
description.beds                          8.39
location.coun

Check for proportion of rows with null values when the lat lon is empty

In [245]:
import pandas as pd
import numpy as np

# Step 1: Calculate the total number of rows
total = len(df)

# Step 2: Calculate the proportion of null entries for all rows
null_proportion_all = df.isnull().sum() / total * 100
sorted_null_proportion_all = round(null_proportion_all.sort_values(ascending=False), 2)

# Step 3: Filter rows where both lat and long are null
both_null_condition = df['location.address.coordinate.lat'].isnull() & df['location.address.coordinate.lon'].isnull()
df_both_null = df[both_null_condition]

# Step 4: Calculate the total number of rows where both lat and long are null
total_both_null = len(df_both_null)

# Step 5: Calculate the proportion of null entries where both lat and long are null
null_proportion_both_null = df_both_null.isnull().sum() / total_both_null * 100
sorted_null_proportion_both_null = round(null_proportion_both_null.sort_values(ascending=False), 2)

# Step 6: Combine the two proportions into a DataFrame for side-by-side comparison
comparison_df = pd.DataFrame({
    'Null Proportion (All Rows)': sorted_null_proportion_all,
    'Null Proportion (Lat & Long Null)': sorted_null_proportion_both_null
})

# Adjust display options to show full DataFrame side by side
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.expand_frame_repr', False)  # Don't break across rows
pd.set_option('display.max_rows', None)  # Show all rows

# Display the DataFrame
print(comparison_df)


                                       Null Proportion (All Rows)  Null Proportion (Lat & Long Null)
branding                                                     0.39                              11.35
community.advertisers                                       99.94                             100.00
community.description.name                                  99.94                             100.00
description.baths                                            2.58                              25.89
description.baths_3qtr                                      93.09                             100.00
description.baths_full                                      10.74                              75.18
description.baths_half                                      72.15                              92.91
description.beds                                             8.39                              75.18
description.garage                                          45.70                          

In [246]:
import pandas as pd
import numpy as np

# Step 1: Filter out rows where both lat and long are null
df_filtered = df[~(df['location.address.coordinate.lat'].isnull() & df['location.address.coordinate.lon'].isnull())]

# Step 2: Fill null list_price values with description.sold_price where description.sold_price is not null
df_filtered['list_price'] = df_filtered['list_price'].fillna(df_filtered['description.sold_price'])

# Step 2: Fill null list_price values with description.sold_price where description.sold_price is not null
df_filtered['list_price'] = df_filtered['description.sold_price'].fillna(df_filtered['list_price'])

# Step 3: Remove rows where list price is null
df_filtered_cleaned = df_filtered[df_filtered['list_price'].notnull()]

# Step 3: Remove rows where list price is null
df_filtered_cleaned = df_filtered[df_filtered['description.sold_price'].notnull()]

# Step 4: Fill null values in description.garage, description.beds, description.bath_x and price_reduced_amount with 0
df_filtered_cleaned['description.garage'] = df_filtered_cleaned['description.garage'].fillna(0)
df_filtered_cleaned['description.beds'] = df_filtered_cleaned['description.beds'].fillna(0)
df_filtered_cleaned['description.baths_full'] = df_filtered_cleaned['description.baths_full'].fillna(0)
df_filtered_cleaned['description.baths_3qtr'] = df_filtered_cleaned['description.baths_3qtr'].fillna(0)
df_filtered_cleaned['description.baths_half'] = df_filtered_cleaned['description.baths_half'].fillna(0)
df_filtered_cleaned['description.baths'] = df_filtered_cleaned['description.baths'].fillna(0)
df_filtered_cleaned['price_reduced_amount'] = df_filtered_cleaned['price_reduced_amount'].fillna(0)

# Step 5: Calculate the total number of rows in the cleaned DataFrame
total_filtered_cleaned = len(df_filtered_cleaned)

# Step 6: Recalculate the proportion of null entries in each column for the cleaned DataFrame
null_proportion_filtered_cleaned = df_filtered_cleaned.isnull().sum() / total_filtered_cleaned * 100

# Step 7: Round and sort the null proportions in descending order
sorted_null_proportion_filtered_cleaned = round(null_proportion_filtered_cleaned.sort_values(ascending=False), 2)

# Display the result
print(sorted_null_proportion_filtered_cleaned)


community.description.name               99.92
source.plan_id                           99.92
community.advertisers                    99.92
source.spec_id                           99.92
flags.is_foreclosure                     99.54
description.sub_type                     82.07
virtual_tours                            81.44
flags.is_price_reduced                   69.42
description.stories                      22.44
description.lot_sqft                     12.73
description.year_built                    8.28
description.sqft                          7.88
photos                                    5.64
primary_photo.href                        5.64
tags                                      4.73
location.county.fips_code                 4.67
products.brand_name                       3.95
list_date                                 3.33
source.agents                             3.33
listing_id                                3.33
source.type                               3.33
flags.is_new_

/var/folders/jj/bww8jnqd2lg73nm4k1z5kbmc0000gn/T/ipykernel_31106/1393002596.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['list_price'] = df_filtered['list_price'].fillna(df_filtered['description.sold_price'])
/var/folders/jj/bww8jnqd2lg73nm4k1z5kbmc0000gn/T/ipykernel_31106/1393002596.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['list_price'] = df_filtered['description.sold_price'].fillna(df_filtered['list_price'])
/var/folders/jj/bww8jnqd2lg73nm4k1z5kbmc0000gn/T/ipyker

In [247]:
print(df_filtered_cleaned['flags.is_foreclosure'].value_counts()) # this is a true false column
print("\n")
print(df_filtered_cleaned['community.description.name'].value_counts()) # this is a suburb description - drop
print("\n")
print(df_filtered_cleaned['source.spec_id'].value_counts()) # this is a random ID column with no analytical value - drop
print("\n")
print(df_filtered_cleaned['description.sub_type'].value_counts()) # this is a type of dwelling - some value here, what do we do?
print("\n")
print(df_filtered_cleaned['virtual_tours'].value_counts()) # these are links to virtual tours on YouTube or other video hosting - should we make this a yes/no column with nulls as no?
print("\n")
print(df_filtered_cleaned['flags.is_price_reduced'].value_counts()) # this is a true false column

flags.is_foreclosure
True    30
Name: count, dtype: int64


community.description.name
Woods of Copper Creek    5
Name: count, dtype: int64


source.spec_id
365-36546-365750000-0016    5
Name: count, dtype: int64


description.sub_type
condo        676
townhouse    496
Name: count, dtype: int64


virtual_tours
[{'type': None, 'href': 'https://cambiomhc.com/our-communities/northbrook/'}]                                                                                                                                                                                                                  20
[{'type': None, 'href': 'https://youtube.com/shorts/2pWsYpUaZ-s?feature=share'}]                                                                                                                                                                                                                5
[{'type': None, 'href': 'https://www.propertypanorama.com/instaview/okc/1090023'}]                          

### Observations: ###
- Almost no entry has a description name, plan_id or spec_id, virtual_tours or 'advertisers' but it's not clear what that would even mean. I want to see what the non-null entries here even look like
- flags.is_foreclosure likely is a yes/no whether this home was repossessed. Of general housing stock having a very small proportion being foreclosures (0.6% here) seems reasonable so I am confident the nulls here mean 'this home wasn't a foreclosure'
- description.baths, baths_full, 3qtr, half:  probably indicate the number of these in the home (as with Garage) so NaNs can be 0
- What about beds being null (8.3%) does this indicate a studio apartment (technically zero bedrooms)? This is an assumption we will make.
- How is sqft being null (10.6%)? replace these with the mean?
- list_price should be removed once we use any list prices to fill null sale prices - we carry forward with sale price as the best indicator of value

### Actions for numerical column: ###
- DROP due to no analytical value: listing_id, property_id, source.plan_id, source.spec_id, location.county.fips_code, location.address.postal_code
- Fillna(0): description.beds, description.garage, description.baths, description.baths_full, .baths_3qtr, .baths_half --> replaced null with 0 as above
- Fillna(0): price_reduced_amount. it is showing how much discount is offered - the % null is identical to flags.is_priced_reduced, so this is the value corresponding to the reduction.

- list_price:                   Fill sale_price with list_price then drop this. Sale Price is a stronger value metric than list, so use S_P and if it is null, we fill it with L_P
- description.year_built:       We can feature engineer this into years since construction
- **description.sold_price:       This column is our prediction variable**
- description.lot_sqft:         ??? Replace with average
- description.sqft:             Replace with average
- Description.stories:          Replace with 1 - no such thing as a zero story home, assume it means bungalow / one story
- location.address.coord.lon + coord.lat   Drop. Coordinates are not relevant in the combined dataset like they may be in specific cities (e.g. North side nicer than South of a given city) and will distort the model.

In [248]:
# Drop unnecessary columns
df_filtered_cleaned = df_filtered_cleaned.drop(columns=[
    'listing_id', 
    'property_id', 
    'source.plan_id', 
    'location.county.fips_code',
    'location.address.postal_code',
    'community.description.name',
    'source.spec_id',
    'community.advertisers',
    'primary_photo.href', 
    'source.agents', 
    'location.street_view_url', 
    'other_listings.rdc', 
    'source.type',
    'permalink',
    'branding',
    'flags.is_new_listing',
    'status' #everything in the dataset was marked as SOLD
]) 

# Encode 'photos' and 'virtual_tours' columns - does it have photos / virtual tours, Yes (1) or No (0)?
df_filtered_cleaned['photos'] = df_filtered_cleaned['photos'].notna().astype(int)
df_filtered_cleaned['virtual_tours'] = df_filtered_cleaned['virtual_tours'].notna().astype(int)

# Convert date columns to a uniform format (YYYY-MM-DD), with error handling for mixed formats
df_filtered_cleaned['last_update_date'] = pd.to_datetime(df_filtered_cleaned['last_update_date'], errors='coerce').dt.strftime('%Y-%m-%d')
df_filtered_cleaned['list_date'] = pd.to_datetime(df_filtered_cleaned['list_date'], errors='coerce').dt.strftime('%Y-%m-%d')
df_filtered_cleaned['description.sold_date'] = pd.to_datetime(df_filtered_cleaned['description.sold_date'], errors='coerce').dt.strftime('%Y-%m-%d')

# Replace all occurrences of 'apartment' with 'condos' in the 'description.type' column (if you see the CSV file you will notice there are some mis-identifed ones as apartments but they are mostly condos... same thing)
df_filtered_cleaned['description.type'] = df_filtered_cleaned['description.type'].replace('apartment', 'condos')

# Remove the 'description.sub_type' column now that the description.type follows the same format (see csv file in data for reference... only subtypes were condos and townhouses)
df_filtered_cleaned = df_filtered_cleaned.drop(columns=['description.sub_type'])

# Replace null values with 0 and encode TRUE as 1 in 'flags.is_foreclosure'
df_filtered_cleaned['flags.is_foreclosure'] = df_filtered_cleaned['flags.is_foreclosure'].fillna(0).replace(True, 1)

# Update 'flags.is_price_reduced' based on 'price_reduced_amount' being greater than 0
df_filtered_cleaned['flags.is_price_reduced'] = (df_filtered_cleaned['price_reduced_amount'] > 0).astype(int)

# Encode 'matterport' where False = 0 and True = 1
df_filtered_cleaned['matterport'] = df_filtered_cleaned['matterport'].replace({False: 0, True: 1})

# Encode 'lead_attributes.show_contact_an_agent' where True = 1 and False = 0
df_filtered_cleaned['lead_attributes.show_contact_an_agent'] = df_filtered_cleaned['lead_attributes.show_contact_an_agent'].replace({False: 0, True: 1})


# Display the updated DataFrame
df_filtered_cleaned.head(5)


/var/folders/jj/bww8jnqd2lg73nm4k1z5kbmc0000gn/T/ipykernel_31106/2505917449.py:38: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_filtered_cleaned['flags.is_foreclosure'] = df_filtered_cleaned['flags.is_foreclosure'].fillna(0).replace(True, 1)
/var/folders/jj/bww8jnqd2lg73nm4k1z5kbmc0000gn/T/ipykernel_31106/2505917449.py:44: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_filtered_cleaned['matterport'] = df_filtered_cleaned['matterport'].replace({False: 0, True: 1})
/var/folders/jj/bww8jnqd2lg73nm4k1z5kbmc0000gn/T/ipykernel_311

,last_update_date,tags,list_date,list_price,photos,virtual_tours,price_reduced_amount,matterport,description.year_built,description.baths_3qtr,description.sold_date,description.sold_price,description.baths_full,description.baths_half,description.lot_sqft,description.sqft,description.baths,description.garage,description.stories,description.beds,description.type,lead_attributes.show_contact_an_agent,flags.is_price_reduced,flags.is_foreclosure,products.brand_name,location.address.state,location.address.coordinate.lon,location.address.coordinate.lat,location.address.city,location.address.state_code,location.address.line,location.county.name
30,2021-04-13,"['central_air', 'central_heat', 'community_out...",2023-12-19,129900.0,0,0,0.0,0,1998.0,0.0,2024-01-16,129900.0,2.0,0.0,11761.0,1478.0,2.0,2.0,1.0,3.0,single_family,1,0,0,essentials,Alabama,-86.178412,32.389075,Montgomery,AL,501 Moorfield Ln,Montgomery
31,2024-01-16,"['city_view', 'hardwood_floors', 'two_or_more_...",2023-11-10,88500.0,1,0,3000.0,0,1945.0,0.0,2024-01-16,88500.0,2.0,0.0,6534.0,1389.0,2.0,1.0,2.0,4.0,single_family,1,1,0,essentials,Alabama,-86.273286,32.382748,Montgomery,AL,2326 Winona Ave,Montgomery
32,2021-04-13,"['carport', 'central_air', 'central_heat', 'di...",2023-12-19,145000.0,0,0,0.0,0,1969.0,0.0,2024-01-16,145000.0,2.0,0.0,17424.0,2058.0,2.0,0.0,1.0,3.0,single_family,1,0,0,essentials,Alabama,-86.221454,32.380023,Montgomery,AL,115 Lookout Ridge Rd,Montgomery
33,2024-01-16,"['carport', 'central_air', 'city_view', 'commu...",2023-07-05,65000.0,1,0,9000.0,0,1955.0,0.0,2024-01-16,65000.0,2.0,0.0,9712.0,1432.0,2.0,0.0,1.0,3.0,single_family,1,1,0,essentials,Alabama,-86.284387,32.386844,Montgomery,AL,611 Glenmore Rd,Montgomery
34,2024-01-12,"['city_view', 'single_story', 'big_yard', 'fen...",2023-10-29,169000.0,1,0,5000.0,0,1984.0,0.0,2024-01-12,169000.0,2.0,0.0,10890.0,1804.0,2.0,0.0,1.0,3.0,single_family,1,1,0,basic_opt_in,Alabama,-86.232662,32.351898,Montgomery,AL,1819 E Trinity Blvd,Montgomery


In [279]:
# verify no nulls in the replaced columns in df_filtered_cleaned
df_filtered_cleaned[['description.baths_full', 'description.baths_3qtr', 'description.baths_half', 'description.baths', 'description.garage', 'description.beds', 'price_reduced_amount']].isnull().sum()

description.baths_full    0
description.baths_3qtr    0
description.baths_half    0
description.baths         0
description.garage        0
description.beds          0
price_reduced_amount      0
dtype: int64

In [250]:
# check in on % nulls
total = len(df_filtered_cleaned)
null_proportion = df_filtered_cleaned.isnull().sum() / total * 100

print(null_proportion.sort_values(ascending=False))

description.stories                      22.441487
description.lot_sqft                     12.727551
description.year_built                    8.275968
list_date                                 8.092397
description.sqft                          7.878232
tags                                      4.726939
products.brand_name                       3.946765
last_update_date                          3.334863
location.address.line                     0.076488
location.address.city                     0.076488
location.county.name                      0.076488
description.sold_price                    0.000000
flags.is_price_reduced                    0.000000
list_price                                0.000000
location.address.state_code               0.000000
photos                                    0.000000
location.address.coordinate.lat           0.000000
location.address.coordinate.lon           0.000000
location.address.state                    0.000000
virtual_tours                  

### Actions for categorical columns ###

- TO DROP: last_update_date, permalink, photos, source.spec_id, primary_photo.href, location.address.line, location.street_view_url, community.advertisers, community.description.name, branding, other_listings.rdc, status
- FILLNA(0): virtual_tours, flags.is_foreclosure, flags.is_price_reduced, matterport
- FILL(other): description.sub_type

- tags:                     More info required
- ~~status                    almost all complete / what are nulls?~~ everything was marked as sold
- list_date                 Take the mean if there is a null? check % of nulls Days since listing could be interesting to find?
- ~~virtual_tours           OHE to 0 - 1 for 'offers virtual tour' or drop. Offering a virtual tour may create more demand and higher bidding for a property --> good to keep in the model~~
- products.brand_name       ??? we should use labelEncoding
- location.address.state    Inclined to drop - different states are not able to be numerically represented --> we should use labelEncoding
- location.address.city     Could be Feature Engineered - find city population and add this as a column? the city is probably important for house price, how to we numerically represent?
- location.county.name      Fill in if city is known
- ~~Description.sub_type~~ --> fillna with standard. condo and townhouse are not insignificant features and a significant number are present so we can dummy variable this.
- ~~flags.is_price_reduced~~ --> contains true and false values, fillna with 0 and encode 0 / 1 (done above)



In [253]:
df_filtered_cleaned.shape # now 32 columns

(6537, 32)

In [254]:
null_proportion = df_filtered_cleaned.isnull().sum() / total * 100

print(null_proportion.sort_values(ascending=False))

description.stories                      22.441487
description.lot_sqft                     12.727551
description.year_built                    8.275968
list_date                                 8.092397
description.sqft                          7.878232
tags                                      4.726939
products.brand_name                       3.946765
last_update_date                          3.334863
location.address.line                     0.076488
location.address.city                     0.076488
location.county.name                      0.076488
description.sold_price                    0.000000
flags.is_price_reduced                    0.000000
list_price                                0.000000
location.address.state_code               0.000000
photos                                    0.000000
location.address.coordinate.lat           0.000000
location.address.coordinate.lon           0.000000
location.address.state                    0.000000
virtual_tours                  

### Dealing with Remaining Columns ###
- description.stories - this is mostly complete with number of stories aside from 22% nulls, assume these are 1.0 storey building
- description.lot_sqft
- description.year_built
- list_date
- description.sqft
- tags - below
- products.brand_name - check uniques and value counts
- last_update_date
- location address city / name

**These changes will be implemented with other column changes / alterations in one executable code block**

In [269]:
df_filtered_cleaned.shape # now 32 columns, 6.5k records

(6537, 32)

In [271]:
# examine the stories column - does it make sense to encode nulls as 1.0 stories?
print(f"Number of stories: {df_filtered_cleaned['description.stories'].unique()})") # default may be single storey, fill with 1.0?

print(f"Number of lot sqft nulls: {df_filtered_cleaned['description.lot_sqft'].isnull().sum()}") # 832 nulls out of 6.5k, fill with average?

print(f"Number of sqft nulls: {df_filtered_cleaned['description.sqft'].isnull().sum()}") # only 515 nulls out of 6.5k, fill with average?

print(f"Range of products.brand_name: {df_filtered_cleaned['products.brand_name'].unique()}") # only 4% null, fill with modal category?

print(f"Number of year_built nulls: {df_filtered_cleaned['description.year_built'].isnull().sum()}") # only 541 nulls out of 6.5k, fill with average?


Number of stories: [ 1.  2. nan  3. 10.  6.  8.  4.])
Number of lot sqft nulls: 832
Number of sqft nulls: 515
Range of products.brand_name: ['essentials' 'basic_opt_in' nan 'advantage_brand' 'advantage_pro']
Number of year_built nulls: 541


### Dealing with Tags

Consider the fact that with tags, there are a lot of categorical variables.
- How many columns would we have if we One Hot Encoding (OHE) tags, city and state?
- Perhaps we can get rid of tags that have a low frequency.

In [ ]:
# OHE categorical variables/ tags here
# tags will have to be done manually

### Dealing with Cities

- Sales will vary drastically between cities and states.  Is there a way to keep information about which city it is without OHE?
- Could we label encode or ordinal encode?  Yes, but this may have undesirable effects, giving nominal data ordinal values.
- What we can do is use our training data to encode the mean sale price by city as a feature (a.k.a. Target Encoding)
    - We can do this as long as we ONLY use the training data - we're using the available data to give us a 'starting guess' of the price for each city, without needing to encode city explicitly
- If you replace cities or states with numerical values (like the mean price), make sure that the data is split so that we don't leak data into the training selection. This is a great time to train test split. Compute on the training data, and join these values to the test data
- Note that you *may* have cities in the test set that are not in the training set. You don't want these to be NA, so maybe you can fill them with the overall mean

In [ ]:
# perform train test split here
# do something with state and city

## Extra Data - STRETCH

> This doesn't need to be part of your Minimum Viable Product (MVP). We recommend you write a functional, basic pipeline first, then circle back and join new data if you have time

> If you do this, try to write your downstream steps in a way it will still work on a dataframe with different features!

- You're not limited to just using the data provided to you. Think/ do some research about other features that might be useful to predict housing prices. 
- Can you import and join this data? Make sure you do any necessary preprocessing and make sure it is joined correctly.
- Example suggestion: could mortgage interest rates in the year of the listing affect the price? 

In [ ]:
# import, join and preprocess new data here

## EDA/ Visualization

Remember all of the EDA that you've been learning about?  Now is a perfect time for it!
- Look at distributions of numerical variables to see the shape of the data and detect outliers.    
    - Consider transforming very skewed variables
- Scatterplots of a numerical variable and the target go a long way to show correlations.
- A heatmap will help detect highly correlated features, and we don't want these.
    - You may have too many features to do this, in which case you can simply compute the most correlated feature-pairs and list them
- Is there any overlap in any of the features? (redundant information, like number of this or that room...)

In [ ]:
# perform EDA here

## Scaling and Finishing Up

Now is a great time to scale the data and save it once it's preprocessed.
- You can save it in your data folder, but you may want to make a new `processed/` subfolder to keep it organized